<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/08_Constraint_Propagation_Arc_Consistency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Constraint Propagation – AC3**

**BACKGROUND**

<p align = 'justify'>The AC3 algorithms allows to tighten the domains of a constraint network so that it is arc consistent. In this exercise you will implement a representation for Boolean Satisfiability problems and the AC3 algorithm to enforce arc consistency.</p>

**Task 1**

<p align = 'justify'>Write a Python class that models a Boolean variable for a constraint satisfaction problem. The class should store a name as string and the current domain with two Booleans indicating if True is part of the domain and if False is part of the domain.</p>

<p align = 'justify'>The domain 𝐷 = {𝑇, 𝐹} should be represented by [True,True], the domain 𝐷 = {𝑇} should be represented by [False,True], the domain 𝐷 = {𝐹} should be represented by [True,False] and the domain 𝐷 = {} should be represented by [False,False].</p>

<p align = 'justify'>The class should have a method GetValues() for getting all possible values from the domain as well as a function RemoveFromDomain(v)for removing a value from the domain.</p>

In [ ]:
import itertools

In [ ]:
class BoolVariable():
    def __init__(self,name):
        self.name_ = name
        self.domain_ = [True,True]

    def RemoveFromDomain(self,v):
        if v:
            self.domain_[1]=False
        else:
            self.domain_[0]=False            
    
    def GetName(self):
        return self.name_
    
    def IsInList(self, list):
        for l in list:
            if self.name_ == l.name_:
                return True
        return False
    
    def GetValues(self):
        values = []
        if self.domain_[0]:
            values.append(False)
        if self.domain_[1]:
            values.append(True)
        return values

**Task 2**

<p align = 'justify'>Write a Python class that models a Boolean disjunction constraint. It should be created by passing a list of positive and negative variables to the constructor. The class should store the scheme, i.e. the variables it operates on, and all combinations of assignments that are valid
for the constraint as list of lists of positive variables (hint: use the function itertools.combinations).</p>

<p align = 'justify'>For example the constraint with positive variables [x1, x2] and negative variable [v3] should represent the constraint 𝑥1 ∨ 𝑥2 ∨ ¬𝑥3 and will result in a scheme [x1,x2,x3]. The list of valid assignments is then [[],[x1], [x2], [x1,x2],[x1,x3], [x2,x3], [x1,x1,x3]].</p> 

<p align = 'justify'>The class should also contain a method CheckConstraint(x,v) for checking if a variable assignment 𝑥𝑖 = 𝑣 could satisfy the constraint taking into account the domains of all other variables in the scheme of the constraint.</p>

<p align = 'justify'>For example, checking (x1,True) should result in True, unless True has been removed from the domain of x1 by calling the function x1.RemoveFromDomain(True).</p>

In [ ]:
class Constraint():
    def __init__(self, positive, negative):
        self.scheme_ = []
        for p in positive:
            self.scheme_.append(p)
        for n in negative:
            self.scheme_.append(n)
        self.combinations_ = self.GetAllCombinations_(positive,negative)
#        self.Print()

    def GetAllCombinations_(self,positive,negative):        
        result = []
        for length in range(len(self.scheme_)+1):
            for subset in itertools.combinations(self.scheme_,length):
                valid = False
                for x in self.scheme_:
                    if x.IsInList(subset):
                        if x.IsInList(positive):
                            valid = True
                            break
                    else: 
                        if x.IsInList(negative):
                            valid = True
                            break
                if valid:
                    result.append(subset)
        return result

    def CheckConstraint(self, x,v):
        result = False            
        for t in self.combinations_:                                
            is_valid = True
            for s in self.scheme_:
                values = s.GetValues()
                if len(values)==1:
                    if not ((values[0] and s.IsInList(t)) or ((not values[0]) and (not s.IsInList(t)))):
                        is_valid = False
            if is_valid:
                if (v and (x.IsInList(t)) or ((not v) and (not x.IsInList(t)))):                                        
                    result = True
                    break
        return result        

    def Print(self):
        for t in self.combinations_:
            for x in self.scheme_:
                if (x.IsInList(t)):
                    print(x.name_, end=",")
                else:
                    print("not",x.name_, end=",")
            print()
        print()

**Task 3**

<p align = 'justify'>Implement a model class to which you can add variables as defined in task 1 and constraints as defined in task 2. Implement the AC3 algorithm to make all domains of all variables in the model arc consistent.
(For the purpose of using this code in a future lab exercise, please make sure that you determine equality of variables based on their names instead of their object references, i.e. for two variables x1 and x2, do not compare them with x1==x2, but rather compare them with x1.name==x2.name)</p>

In [ ]:
class Model():
    def __init__(self):
        self.variables_ = []
        self.constraints_ = []
        
    def AddBoolVariable(self, name):
        v = BoolVariable(name)
        self.variables_.append(v)
        return v
        
    def AddOrConstraint(self, positive, negative):
        c = Constraint(positive,negative)
        self.constraints_.append(c)

    def PrintDomains(self):
        for x in self.variables_:
            print(x.GetName(), x.GetValues())
        print()

    def Revise_(self,x,c):
        change = False
        for v in x.GetValues():
            if not c.CheckConstraint(x,v):
                x.RemoveFromDomain(v)
                change = True
        return change

    def AC3(self):
        Q = []
        for x in self.variables_:
            for c in self.constraints_:
                if x.IsInList(c.scheme_):
                    Q.append((x,c))
        while len(Q)>0:
            (x,c) = Q[0]
            Q.remove(Q[0])
            if (self.Revise_(x,c)):
                if len(x.GetValues())==0:
                    return False
                else:
                    for c2 in self.constraints_:
                        if (c2 != c) and (x.IsInList(c2.scheme_)):
                            for x2 in c2.scheme_:
                                if not (x2,c2) in Q:
                                    Q.append((x2,c2))
        return True

In [ ]:
model = Model()

In [ ]:
x1 = model.AddBoolVariable("x1")
x2 = model.AddBoolVariable("x2")
x3 = model.AddBoolVariable("x3")
x4 = model.AddBoolVariable("x4")

In [ ]:
model.AddOrConstraint([x1,x2],[])
model.AddOrConstraint([],[x2])
model.AddOrConstraint([x3],[x1])
model.AddOrConstraint([x4],[x2])

In [ ]:
model.PrintDomains()    
model.AC3()
model.PrintDomains()

x1 [False, True]
x2 [False, True]
x3 [False, True]
x4 [False, True]

x1 [True]
x2 [False]
x3 [True]
x4 [False, True]

